In [1]:
import numpy as np
from astropy.coordinates import SkyCoord, Angle, EarthLocation, Longitude, spherical_to_cartesian, cartesian_to_spherical
from astropy.time import Time, TimeDelta
from astropy import units

In [2]:
ra_hrs = 12.1
dec_degs = -42.3
mjd = 55780.1
latitude = Angle('-26d42m11.94986s')
longitude = Angle('116d40m14.93485s')

In [3]:
print(latitude.rad)
print(longitude.rad)

-0.466060844839
2.03628986686


In [4]:
obs_time = Time(mjd, format='mjd', location = (longitude, latitude))
lst_mean = obs_time.sidereal_time('mean')
lst_apparent = obs_time.sidereal_time('apparent')
print(lst_mean.rad)
print(lst_apparent.rad)

1.88387679923
1.88395723805


In [5]:
(lst_mean - lst_apparent).to('deg')

<Angle -0.0046088 deg>

In [6]:
mwa_tools_lst_mean = np.float64(1.8838984)

In [7]:
print(Angle(lst_mean.rad - mwa_tools_lst_mean, unit='rad').to('deg'))
print(Angle(lst_apparent.rad - mwa_tools_lst_mean, unit='rad').to('deg'))

-0d00m04.4555s
0d00m12.1362s


In [8]:
print((lst_mean - Angle(mwa_tools_lst_mean, unit='rad')).to('deg'))
print((lst_apparent - Angle(mwa_tools_lst_mean, unit='rad')).to('deg'))

-0d00m04.4555s
0d00m12.1362s


In [9]:
new_time = obs_time + TimeDelta('-0.8' * units.s)
print(new_time.sidereal_time('apparent').rad)
(new_time.sidereal_time('apparent') - Angle(mwa_tools_lst_mean, unit='rad')).to('deg')

1.88389890112


<Angle 2.87119341e-05 deg>

# -> mean LST disagrees by ~4.5 arcseconds (should we use apparent??)

In [10]:
icrs_coord = SkyCoord(ra=Angle(ra_hrs, unit='hr'), dec=Angle(dec_degs, unit='deg'),
                      obstime=Time(mjd, format='mjd'))
icrs_coord

<SkyCoord (ICRS): (ra, dec) in deg
    (181.5, -42.3)>

In [11]:
icrs_coord.ra.rad, icrs_coord.dec.rad

(3.1677725923697078, -0.7382742735936013)

In [12]:
gcrs_coord = icrs_coord.transform_to('gcrs')
gcrs_coord

<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec) in deg
    (181.49525535, -42.3015925)>

In [13]:
gcrs_coord.ra.rad, gcrs_coord.dec.rad

(3.167689782620545, -0.7383020680162412)

In [14]:
# This is the change in ra & dec between icrs & gcrs
print(icrs_coord.ra-gcrs_coord.ra)
print(icrs_coord.dec-gcrs_coord.dec)

0d00m17.0807s
0d00m05.733s


In [15]:
hour_angle_mean = lst_mean.rad - gcrs_coord.ra.rad
hour_angle_apparent = lst_apparent.rad - gcrs_coord.ra.rad

print(hour_angle_mean)
print(hour_angle_apparent)

-1.28381298339
-1.28373254457


In [16]:
# mwa_tools comp2
# It turns out these are not actually used in the return values at all. So don't worry about them
mwa_tools_comp2_ra = 3.1703987
mwa_tools_comp2_dec = -0.73946297

In [17]:
# compare mwa_tools comp2 to gcrs
print(Angle(gcrs_coord.ra.rad - mwa_tools_comp2_ra, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))
print(Angle(gcrs_coord.dec.rad - mwa_tools_comp2_dec, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))

-0deg09m18.7543s
0deg03m59.4532s


In [18]:
# mwa_tools comp3
mwa_tools_comp3_ra = 3.1676898
mwa_tools_comp3_dec = -0.73830205

In [19]:
# compare mwa_tools comp3 to gcrs
print(Angle(gcrs_coord.ra.rad - mwa_tools_comp3_ra, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))
print(Angle(gcrs_coord.dec.rad - mwa_tools_comp3_dec, unit='rad').to_string(unit=units.degree, sep=('deg', 'm', 's')))

-0deg00m00.0036s
-0deg00m00.0037s


# -> mwa_tools ra_aber/dec_aber (calc3) agrees with GCRS, meaning that it accounts for aberration but not nutation or precession

In [20]:
# The frame radio astronomers call the apparent or current epoch is the
# "true equator & equinox" frame, notated E_gamma in the USNO circular
# astropy doesn't have this frame but it's pretty easy to adapt the CIRS frame
# by modifying the ra to reflect the difference between
# GAST (Grenwich Apparent Sidereal Time) and the earth rotation angle (theta)
def egamma_to_cirs_ra(egamma_ra, time):
    from astropy import _erfa as erfa
    from astropy.coordinates.builtin_frames.utils import get_jd12
    era = erfa.era00(*get_jd12(time, 'ut1'))
    theta_earth = Angle(era, unit='rad')

    assert(isinstance(time, Time))
    gast = time.sidereal_time('apparent', longitude=0)
    cirs_ra = egamma_ra - (gast-theta_earth)
    return cirs_ra

In [21]:
# now get equivilents of lst, lat in GCRS (rather than precessed) frame:
# effectively current zenith "unprecessed" to J2000 but not undoing aberration
# mwa_tools calls these "lmst2000" and "newarrlat"
# also need gcrs hour angle (ha_gcrs), what they call "ha2000" which is ("lmst2000" - apparent ra)

loc_obj = EarthLocation.from_geodetic(lon=longitude, lat=latitude)

# use CIRS but with a different ra to account for the difference between LST & earth's rotation angle
cirs_ra = egamma_to_cirs_ra(obs_time.sidereal_time('apparent'), obs_time)

egamma_zenith_coord = SkyCoord(ra=cirs_ra, dec=latitude, frame='cirs',
                               obstime=obs_time, location = loc_obj)
egamma_zenith_coord

<SkyCoord (CIRS: obstime=55780.1): (ra, dec) in deg
    (107.78961213, -26.70331941)>

In [22]:
# check where it is in altaz (should be near zenith):
egamma_zenith_altaz = egamma_zenith_coord.transform_to('altaz')
egamma_zenith_altaz

<SkyCoord (AltAz: obstime=55780.1, location=(-2559302.5737783727, 5095070.526830904, -2848887.400942108) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0, obswl=1.0 micron): (az, alt) in deg
    (28.60732437, 89.99985797)>

In [23]:
(egamma_zenith_altaz.alt - Angle('90d')).to('deg')

<Angle -0.00014203 deg>

In [24]:
gcrs_zenith = egamma_zenith_altaz.transform_to('gcrs')
gcrs_zenith

<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec) in deg
    (107.82139279, -26.68249473)>

In [25]:
print(gcrs_zenith.ra.rad)
print(gcrs_zenith.dec.rad)

1.88183830833
-0.465697385751


In [26]:
# This is the change between egamma & gcrs
print(gcrs_zenith.ra - lst_apparent)
print(gcrs_zenith.dec - egamma_zenith_coord.dec)

-0d07m17.0606s
0d01m14.9688s


In [27]:
ha_gcrs = Longitude(gcrs_zenith.ra - gcrs_coord.ra)
ha_gcrs

<Longitude 286.32613744 deg>

In [28]:
ha_gcrs.rad

4.997333832891704

In [29]:
# for these, I used the apparent lst (and aberration corrected ra/decs) from this notebook,
# so that's not a source of error
mwa_tools_comp4_newarrlat = -0.46569739
mwa_tools_comp4_lmst2000 = 1.8818386
mwa_tools_comp4_ha2000 = 4.9973341

In [30]:
# This is what the MWA tools code gives as the change between precessed & not
print(Angle(lst_apparent.rad - mwa_tools_comp4_lmst2000, unit='rad').to('deg'))
print(Angle(latitude.rad - mwa_tools_comp4_newarrlat, unit='rad').to('deg'))

0d07m17.0005s
-0d01m14.9679s


In [31]:
print(Angle(gcrs_zenith.ra.rad - mwa_tools_comp4_lmst2000, unit='rad').to('deg'))
print(Angle(gcrs_zenith.dec.rad - mwa_tools_comp4_newarrlat, unit='rad').to('deg'))
print(Angle(ha_gcrs.rad - mwa_tools_comp4_ha2000, unit='rad').to('deg'))

-0d00m00.0602s
0d00m00.0009s
-0d00m00.0551s


In [32]:
# redo using the mean lst that matches mwa_tools
new_cirs_ra = egamma_to_cirs_ra(new_time.sidereal_time('apparent'), new_time)

loc_obj = EarthLocation.from_geodetic(lon=longitude, lat=latitude)
new_egamma_zenith_coord = SkyCoord(ra=new_cirs_ra, dec=latitude, frame='cirs',
                                   obstime=new_time, location = loc_obj)
print(new_egamma_zenith_coord)
new_egamma_zenith_altaz = new_egamma_zenith_coord.transform_to('altaz')
print(new_egamma_zenith_altaz)
print((new_egamma_zenith_altaz.alt - Angle('90d')).to('deg'))

new_gcrs_zenith = new_egamma_zenith_altaz.transform_to('gcrs')
new_ha_gcrs = Longitude(new_gcrs_zenith.ra - gcrs_coord.ra)

new_mwa_tools_comp4_ha2000 = 4.9972758
new_mwa_tools_comp4_newarrlat = -0.46569745
new_mwa_tools_comp4_lmst2000 = 1.8817802

print(Angle(new_gcrs_zenith.ra.rad - new_mwa_tools_comp4_lmst2000, unit='rad').to('deg'))
print(Angle(new_gcrs_zenith.dec.rad - new_mwa_tools_comp4_newarrlat, unit='rad').to('deg'))
print(Angle(new_ha_gcrs.rad - new_mwa_tools_comp4_ha2000, unit='rad').to('deg'))


<SkyCoord (CIRS: obstime=55780.0999907): (ra, dec) in deg
    (107.78626967, -26.70331941)>
<SkyCoord (AltAz: obstime=55780.0999907, location=(-2559302.5737783727, 5095070.526830904, -2848887.400942108) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0, obswl=1.0 micron): (az, alt) in deg
    (28.60732375, 89.99985797)>
-0d00m00.5113s
-0d00m00.0449s
-0d00m00s
-0d00m00.0605s


# => Egamma -> CIRS -> CGRS agrees with computation4

In [33]:
# An undocumented feature lets you get the rotation matrix for some Astropy transforms
from astropy import coordinates
from astropy.coordinates.builtin_frames import icrs_fk5_transforms
fk5 = coordinates.FK5()
icrs_fk5_transforms.icrs_to_fk5(None, fk5)

array([[ 1.00000000e+00, -1.11022333e-07, -4.41180450e-08],
       [ 1.11022337e-07,  1.00000000e+00,  9.64779225e-08],
       [ 4.41180343e-08, -9.64779274e-08,  1.00000000e+00]])

In [34]:
# # But not for the ones we want -- this errors:
# from astropy import coordinates
# from astropy.coordinates.builtin_frames import icrs_cirs_transforms
# cirs = coordinates.CIRS(obstime=Time(mjd, format='mjd'))
# icrs_cirs_transforms.icrs_to_cirs(None, cirs)

In [35]:
# now define an antenna position to work on uvw rotation
# the positions used in the mwa_tools code are the same as those used in uvfits antenna positions:
# relative to the array center (expressed in ITRS), but rotated so that the x axis goes through the meridian

from pyuvdata import utils as uvutils
array_center_xyz = np.array([-2559454.08, 5095372.14, -2849057.18])

# in east/north/up frame (relative to array center) in meters:
x_ant = -101.94
y_ant = 0156.41
z_ant = 0001.24

lat_lon_alt = uvutils.LatLonAlt_from_XYZ(array_center_xyz)
ant_xyz_abs = uvutils.ECEF_from_ENU(np.array([x_ant, y_ant, z_ant]), lat_lon_alt[0], lat_lon_alt[1], lat_lon_alt[2])

ant_xyz_rel_itrs = ant_xyz_abs - array_center_xyz
ant_xyz_rel_rot = uvutils.rotECEF_from_ECEF(ant_xyz_rel_itrs, lat_lon_alt[1])

In [36]:
print(ant_xyz_rel_rot)

[  71.39382794 -101.94        139.17092739]


In [37]:
egamma_zenith_xyz = spherical_to_cartesian(1, egamma_zenith_coord.dec, egamma_zenith_coord.ra)
egamma_zenith_xyz

(<Quantity -0.27293726>, <Quantity 0.85062987>, <Quantity -0.44937075>)

In [38]:
egamma_ant_spherical = cartesian_to_spherical(ant_xyz_rel_rot[0], ant_xyz_rel_rot[1], ant_xyz_rel_rot[2])
egamma_ant_spherical

(<Quantity 186.70133717>,
 <Latitude 0.84116482 rad>,
 <Longitude 5.32335079 rad>)

In [39]:
egamma_zenith_cartesian = SkyCoord(x=egamma_zenith_xyz[0], y=egamma_zenith_xyz[1], z=egamma_zenith_xyz[2],
                                   representation = 'cartesian', frame='cirs',
                                   obstime=obs_time)

In [40]:
egamma_zenith_cartesian.representation='spherical'
egamma_zenith_cartesian

<SkyCoord (CIRS: obstime=55780.1): (ra, dec, distance) in (deg, deg, )
    (107.78961213, -26.70331941, 1.)>

In [41]:
print(egamma_zenith_cartesian.ra - egamma_zenith_coord.ra)
print(egamma_zenith_cartesian.dec - egamma_zenith_coord.dec)

0d00m00s
0d00m00s


In [42]:
cirs_ant_ra = egamma_to_cirs_ra(egamma_ant_spherical[2], obs_time)

cirs_ant_xyz = spherical_to_cartesian(egamma_ant_spherical[0], egamma_ant_spherical[1], cirs_ant_ra)
cirs_ant_xyz

(<Quantity 71.12102561>, <Quantity -102.13051446>, <Quantity 139.17092739>)

In [43]:
cirs_ant_coord = SkyCoord(x=cirs_ant_xyz[0]*units.m, y=cirs_ant_xyz[1]*units.m, z=cirs_ant_xyz[2]*units.m,
                          representation = 'cartesian', frame='cirs',
                          obstime=obs_time)
cirs_ant_coord

<SkyCoord (CIRS: obstime=55780.1): (x, y, z) in m
    (71.12102561, -102.13051446, 139.17092739)>

In [44]:
gcrs_ant_coord = cirs_ant_coord.transform_to('gcrs')
gcrs_ant_coord.representation = 'cartesian'
gcrs_ant_coord

<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (x, y, z) in m
    (71.28260457, -102.13175108, 139.08732594)>

In [45]:
mwa_tools_precxyz_xprec = 71.286688
mwa_tools_precxyz_yprec = -102.13304
mwa_tools_precxyz_zprec = 139.08429

In [46]:
print(mwa_tools_precxyz_xprec - ant_xyz_rel_rot[0])
print(mwa_tools_precxyz_yprec - ant_xyz_rel_rot[1])
print(mwa_tools_precxyz_zprec - ant_xyz_rel_rot[2])

-0.1071399426444799
-0.1930399991369427
-0.0866373932501574


In [47]:
print((gcrs_ant_coord.x - mwa_tools_precxyz_xprec*units.m).to('mm'))
print((gcrs_ant_coord.y - mwa_tools_precxyz_yprec*units.m).to('mm'))
print((gcrs_ant_coord.z - mwa_tools_precxyz_zprec*units.m).to('mm'))
# these differences might be due to using a different lst (apparent)
# or using relative rather than absolute locations

-4.08343381373 mm
1.28892441548 mm
3.03593942368 mm


# The precessed positions match to < 5mm

In [48]:
# try something else. Define antennas in ITRS frame. Then rotate them to GCRS
array_center_coord = SkyCoord(x=array_center_xyz[0]*units.m,
                              y=array_center_xyz[1]*units.m,
                              z=array_center_xyz[2]*units.m,
                              representation = 'cartesian', frame='itrs', obstime=obs_time)
print(array_center_coord)
itrs_coord = SkyCoord(x=ant_xyz_abs[0]*units.m, y=ant_xyz_abs[1]*units.m, z=ant_xyz_abs[2]*units.m,
                      representation = 'cartesian', frame='itrs', obstime=obs_time)
itrs_coord

<SkyCoord (ITRS: obstime=55780.1): (x, y, z) in m
    (-2559454.08, 5095372.14, -2849057.18)>


<SkyCoord (ITRS: obstime=55780.1): (x, y, z) in m
    (-2559395.03251243, 5095481.69471912, -2848918.00907261)>

In [49]:
print(itrs_coord.x - array_center_coord.x)
print(itrs_coord.y - array_center_coord.y)
print(itrs_coord.z - array_center_coord.z)

59.0474875653 m
109.554719117 m
139.170927393 m


In [50]:
gcrs_array_center = array_center_coord.transform_to('gcrs')
print(gcrs_array_center)
gcrs_from_itrs_coord = itrs_coord.transform_to('gcrs')
gcrs_from_itrs_coord

<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, m)
    (107.82119295, -26.52844311, 6374225.38409199)>


<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, m)
    (107.82016689, -26.52703835, 6374227.04770571)>

In [51]:
gcrs_array_center.representation = 'cartesian'
print(gcrs_array_center)
gcrs_from_itrs_coord.representation = 'cartesian'
gcrs_from_itrs_coord

<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (x, y, z) in m
    (-1745419.50769061, 5429444.57475286, -2846996.94398546)>


<SkyCoord (GCRS: obstime=55780.1, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (x, y, z) in m
    (-1745344.09283064, 5429543.69894979, -2846857.85990781)>

In [52]:
gcrs_rel = (np.array([gcrs_from_itrs_coord.x.value, gcrs_from_itrs_coord.y.value, gcrs_from_itrs_coord.z.value])
            - np.array([gcrs_array_center.x.value, gcrs_array_center.y.value, gcrs_array_center.z.value]))

print(gcrs_rel[0]*units.m)
print(gcrs_rel[1]*units.m)
print(gcrs_rel[2]*units.m)

75.4148599696 m
99.1241969327 m
139.084077647 m


In [53]:
gcrs_lat_lon_alt = uvutils.LatLonAlt_from_XYZ(np.array([gcrs_array_center.x.value,
                                                        gcrs_array_center.y.value,
                                                        gcrs_array_center.z.value]))
gcrs_rel_rot = uvutils.rotECEF_from_ECEF(gcrs_rel, gcrs_lat_lon_alt[1])

print(gcrs_rel_rot[0]*units.m)
print(gcrs_rel_rot[1]*units.m)
print(gcrs_rel_rot[2]*units.m)

71.2873201004 m
-102.132881653 m
139.084077647 m


In [54]:
print((gcrs_rel_rot[0]*units.m - gcrs_ant_coord.x).to('mm'))
print((gcrs_rel_rot[1]*units.m - gcrs_ant_coord.y).to('mm'))
print((gcrs_rel_rot[2]*units.m - gcrs_ant_coord.z).to('mm'))

4.7155342145 mm
-1.13057712116 mm
-3.24829200218 mm


In [55]:
print(((gcrs_rel_rot[0] - mwa_tools_precxyz_xprec)*units.m).to('mm'))
print(((gcrs_rel_rot[1] - mwa_tools_precxyz_yprec)*units.m).to('mm'))
print(((gcrs_rel_rot[2] - mwa_tools_precxyz_zprec)*units.m).to('mm'))

0.632100400765 mm
0.158347294317 mm
-0.212352578501 mm


# Starting from ITRS does even better: positions match to < 1mm